# Assignment 3 (group B3)


This file is the solution proposed for assignment 3 by group B3.

### Part 1

The goal of this part is to retrieve information of search results in Wallapop for “Bicicleta de carretera”, “Bicicleta plegable” and “MTB”.

<div class="alert alert-info">Import all ncesessary libraries.</div>

In [1]:
from selenium import webdriver
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
import time
import bs4
import re
import pandas as pd
import requests
from datetime import datetime
import locale
import numpy as np

<div class="alert alert-info">First, we create a new variable called <b>PATH</b> to store the location of the chrome driver.</div>

In [2]:
PATH = '/Users/inakirandezciordia/Desktop/Cloud Computing/chromedriver'

<div class="alert alert-info">Then we initialize the webdriver and access the Wallapop webpage</div>

In [3]:
driver = webdriver.Chrome(PATH)
driver.get("https://www.wallapop.com/")

/var/folders/0t/zj20906d0wgb0wjbz0xl310h0000gn/T/ipykernel_99520/3921612051.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(PATH)


<div class="alert alert-info">We accept the cookies of the webpage</div>

In [4]:
cookies = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//button[contains(text(), "Confirmar preferencias sugeridas")]')))
cookies.click()

<div class="alert alert-info">We search “bicicleta” in Wallapop</div>

In [4]:
busca_bicicleta = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[name='keywords']")))
busca_bicicleta.clear()
busca_bicicleta.send_keys("bicicleta")
Bicicleta = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="searchBoxForm"]/div/div[3]/ul/li[1]')))
Bicicleta.click()

<div class="alert alert-info">Change location to "Barcelona, España" and slect distance = 10 km</div>

In [5]:
location = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '/html/body/tsl-root/tsl-public/div/div/tsl-search/div/div/tsl-filters-wrapper/div/div/tsl-filter-group/div/div[2]/tsl-filter-host/div/tsl-location-filter/tsl-filter-template/div/tsl-bubble/div/div'))).click()
location_barcelona = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "/html/body/tsl-root/tsl-public/div/div/tsl-search/div/div/tsl-filters-wrapper/div/div[2]/tsl-filter-group/div/div[2]/tsl-filter-host/div/tsl-location-filter/tsl-filter-template/div/div/div[2]/div/tsl-drawer-placeholder-template/div/div/div[1]/input")))
location_barcelona.clear()
location_barcelona.send_keys("Barcelona")
Barcelona = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="ngb-typeahead-1-0"]')))
Barcelona.click()

In [6]:
move = ActionChains(driver)
wait = WebDriverWait(driver, 0.5)
slider = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '/html/body/tsl-root/tsl-public/div/div/tsl-search/div/div/tsl-filters-wrapper/div/div[2]/tsl-filter-group/div/div[2]/tsl-filter-host/div/tsl-location-filter/tsl-filter-template/div/div/div[2]/div/tsl-drawer-placeholder-template/div/div/form/tsl-slider-form/form/ngx-slider/span[5]')))
find_elem = None
scroll = 0
scroll_limit = 3000
while not find_elem and scroll > -500:
    move.click_and_hold(slider).move_by_offset(-50, 0).release().perform()
    scroll += -50
    try:
        find_elem = wait.until(EC.text_to_be_present_in_element((By.XPATH, "/html/body/tsl-root/tsl-public/div/div/tsl-search/div/div/tsl-filters-wrapper/div/div[2]/tsl-filter-group/div/div[2]/tsl-filter-host/div/tsl-location-filter/tsl-filter-template/div/div/div[2]/div/tsl-drawer-placeholder-template/div/div/form/tsl-slider-form/form/ngx-slider/span[9]"), '10km'))
    except:
        pass
aplicar = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//button[contains(text(), "Aplicar")]')))
aplicar.click()

<div class="alert alert-info">Limit price to 800€</div>

In [7]:
precio = driver.find_element(By.XPATH, '/html/body/tsl-root/tsl-public/div/div/tsl-search/div/div/tsl-filters-wrapper/div/div/tsl-filter-group/div/div[3]').click()
precio800 = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "/html/body/tsl-root/tsl-public/div/div/tsl-search/div/div/tsl-filters-wrapper/div/div[2]/tsl-filter-group/div/div[3]/tsl-filter-host/div/tsl-range-filter/tsl-filter-template/div/div/div[2]/div/form/div/div[2]/input")))
precio800.clear()
precio800.send_keys("800")
time.sleep(3)
aplicar = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '/html/body/tsl-root/tsl-public/div/div/tsl-search/div/div/tsl-filters-wrapper/div/div[2]/tsl-filter-group/div/div[3]/tsl-filter-host/div/tsl-range-filter/tsl-filter-template/div/div/div[3]/tsl-button[2]')))
aplicar.click()

<div class="alert alert-info">Create two lists to later narrow down the search to include only results in the “Bicicletas y triciclos” subfield and select labels “Bicicletas de carretera”, “Bicicletas plegables” and “MTB” and states 'nuevo', 'como nuevo' and 'en buen estado'</div>

In [8]:
suboptions = ["plegables", "carretera", "MTB"]
estados = ['nuevo', 'como nuevo', 'en buen estado']
estados_xpath = {'nuevo': '/html/body/tsl-root/tsl-public/div/div/tsl-search/div/div/tsl-filters-wrapper/div/div[2]/tsl-filter-group/div/div[5]/tsl-filter-host/div/tsl-multi-select-filter/div/tsl-filter-template/div/div/div[2]/div/tsl-drawer-placeholder-template/div/div/form/tsl-multi-select-form/div[1]/tsl-multi-select-option/div/tsl-checkbox-form',
          'como nuevo':'/html/body/tsl-root/tsl-public/div/div/tsl-search/div/div/tsl-filters-wrapper/div/div[2]/tsl-filter-group/div/div[5]/tsl-filter-host/div/tsl-multi-select-filter/div/tsl-filter-template/div/div/div[2]/div/tsl-drawer-placeholder-template/div/div/form/tsl-multi-select-form/div[2]/tsl-multi-select-option/div/tsl-checkbox-form',
          'en buen estado':'/html/body/tsl-root/tsl-public/div/div/tsl-search/div/div/tsl-filters-wrapper/div/div[2]/tsl-filter-group/div/div[5]/tsl-filter-host/div/tsl-multi-select-filter/div/tsl-filter-template/div/div/div[2]/div/tsl-drawer-placeholder-template/div/div/form/tsl-multi-select-form/div[3]/tsl-multi-select-option/div/tsl-checkbox-form'}

<div class="alert alert-info">Now we will define several functions that will be needed inside a loop to extract all the data.</div>

<div class="alert alert-warning">Define a function that will select the subcategories one by one and applies the selection. This function will be called a few cells later.</div>

In [9]:
def select_subcategoria_and_estado(subcategoria, estado, est_xpath):
    #click subcategoria
    subcat = driver.find_element(By.XPATH, '/html/body/tsl-root/tsl-public/div/div/tsl-search/div/div/tsl-filters-wrapper/div/div/tsl-filter-group/div/div[4]/tsl-filter-host/div/tsl-multi-select-filter/div/tsl-filter-template/div/tsl-bubble/div').click()
    #click bicicletas y triciclos
    bici_y_tri = driver.find_element(By.XPATH, "//*[contains(text(), ' Bicicletas y triciclos ')]").click()
    #select type of bike
    sub_options = driver.find_elements(By.CLASS_NAME, 'MultiSelectOption')
    for i in range(len(sub_options)):
        if subcategoria in sub_options[i].text:
            driver.find_element(By.XPATH, '/html/body/tsl-root/tsl-public/div/div/tsl-search/div/div/tsl-filters-wrapper/div/div[2]/tsl-filter-group/div/div[4]/tsl-filter-host/div/tsl-multi-select-filter/div/tsl-filter-template/div/div/div[2]/div/tsl-drawer-placeholder-template/div/div/form/tsl-multi-select-form/div/div[2]/div[' + str(i+1) + ']/tsl-multi-select-option/div/tsl-checkbox-form/label').click()
    #apply selection
    aplicar = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '/html/body/tsl-root/tsl-public/div/div/tsl-search/div/div/tsl-filters-wrapper/div/div[2]/tsl-filter-group/div/div[4]/tsl-filter-host/div/tsl-multi-select-filter/div/tsl-filter-template/div/div/div[3]/tsl-button[2]')))
    aplicar.click()
    
    #click estado
    estado_select = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '/html/body/tsl-root/tsl-public/div/div/tsl-search/div/div/tsl-filters-wrapper/div/div/tsl-filter-group/div/div[5]/tsl-filter-host/div/tsl-multi-select-filter/div/tsl-filter-template/div/tsl-bubble'))).click()
    #select estado
    WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, est_xpath[estado]))).click()
    #apply selection
    aplicar = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '/html/body/tsl-root/tsl-public/div/div/tsl-search/div/div/tsl-filters-wrapper/div/div[2]/tsl-filter-group/div/div[5]/tsl-filter-host/div/tsl-multi-select-filter/div/tsl-filter-template/div/div/div[3]/tsl-button[2]')))
    aplicar.click()

<div class="alert alert-warning">Define a function that will load more products in the webpage. This function will be called a few cells later.</div>

In [10]:
def load_more_products():
    try:
        #click "ver más productos"
        time.sleep(3)
        aplicar = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="btn-load-more"]')))
        aplicar.click()
    except:
        #scroll down
        for i in range(8):
            driver.execute_script("window.scrollBy(0,5000)")
            time.sleep(3)

<div class="alert alert-warning">Define a function that will deselect the previous suboptions to be able to start over the loop. This function will be called a few cells later.</div>

In [11]:
def deselect_options():
    #deselect estado
    driver.find_element(By.XPATH, '/html/body/tsl-root/tsl-public/div/div/tsl-search/div/div/tsl-filters-wrapper/div/div/tsl-filter-group/div/div[5]/tsl-filter-host/div/tsl-multi-select-filter/div/tsl-filter-template/div/tsl-bubble/div/div[2]').click()
    time.sleep(2)
    #deselect type of bike
    driver.find_element(By.XPATH, '/html/body/tsl-root/tsl-public/div/div/tsl-search/div/div/tsl-filters-wrapper/div/div/tsl-filter-group/div/div[4]/tsl-filter-host/div/tsl-multi-select-filter/div/tsl-filter-template/div/tsl-bubble/div/div[2]').click()
    time.sleep(2)

<div class="alert alert-warning">Define a function that will delete all the non ascii letters. This function will be necessary when creating the url of each advertisement.</div>

In [12]:
def remove_accents(raw_text):
    
    """This function removes non ascii letters."""
    
    raw_text = raw_text.lower()

    raw_text = re.sub(u"[àáâãäå]", 'a', raw_text)
    raw_text = re.sub(u"[èéêë]", 'e', raw_text)
    raw_text = re.sub(u"[ìíîï]", 'i', raw_text)
    raw_text = re.sub(u"[òóôõö]", 'o', raw_text)
    raw_text = re.sub(u"[ùúûü]", 'u', raw_text)
    raw_text = re.sub(u"[ýÿ]", 'y', raw_text)
    raw_text = re.sub(u"[ß]", 'ss', raw_text)
    raw_text = re.sub(u"[ñ]", 'n', raw_text)
    raw_text = re.sub(u"[-,/)(.ø🚲☺️·&|!?¡¿”]", ' ', raw_text)
    raw_text = re.sub(u"[ºª]", '', raw_text)
    raw_text = re.sub(u'["]', ' ', raw_text)
    raw_text = re.sub(u"['']", ' ', raw_text)
    raw_text = re.sub(u'[+]', '-', raw_text)
    raw_text = re.sub(' +', ' ', raw_text)
    raw_text = raw_text.rstrip().lstrip()
    return raw_text

<div class="alert alert-warning">Define a function that will gather all the information from the html soup and store it in lists. This function will be called a few cells later.</div>

In [13]:
def get_info_listing(sopa):
    locale.setlocale(locale.LC_TIME, 'es_ES')
    fechas = []
    link_img = []
    titulo_anuncio = []
    precio_anuncio = []
    descripcion_anuncio = []
    link_anuncio = []
    estado_articulo = []
    tipo_articulo = []
    base_url="https://es.wallapop.com/item/"

    for bikes in soup.find_all("a", {"class": "ItemCardList__item ng-star-inserted"}):
        #stop the loop if we already have 250 listings
        if len(titulo_anuncio) > 249:
            break
        else:
            imagen = bikes.find("img", {"class": "w-100 h-100"})['src']
            title_ad = bikes.find("p", {"class": "ItemCard__title my-1 ItemCard__title--with-description"}).text.rstrip().lstrip()
            precio = bikes.find("span", {"class": "ItemCard__price ItemCard__price--bold"}).text[1:-2]
            id_del_articulo = imagen.split("c10420p")[1].split("/")[0]
            titulo_sin_acento = remove_accents(title_ad).split(" ")
            anuncio = base_url + str("-".join(titulo_sin_acento)) + "-" + id_del_articulo

            info_articulo = requests.get(anuncio)
            if info_articulo.status_code == 200: #if the url is correct, gather its information
                try:
                    sopas = bs4.BeautifulSoup(info_articulo.text, 'html.parser')
                    fecha = sopas.find('div', {"class": 'card-product-detail-user-stats-published'})
                    fecha = re.sub(u'[\n\t]', '', fecha.text)
                    # convert date to a DateTime object
                    datetime_object = datetime.strptime(fecha, '%d-%b-%Y')
                    tipo = sopas.find_all('span', {"class": 'ExtraInfo__taxonomyBubble__text'})[1].text
                    estado = sopas.find('span', {"class": 'ExtraInfo__text'}).text

                    #append all the information
                    fechas.append(datetime_object)
                    tipo_articulo.append(tipo)
                    estado_articulo.append(estado)
                    link_img.append(imagen)
                    titulo_anuncio.append(title_ad)
                    precio_anuncio.append(float(re.sub(u"[,]", '.', precio)))
                    descripcion_anuncio.append(bikes.find("p", {"class": "ItemCard__description ItemCard__description--light mb-0 ng-star-inserted"}).text[1:-1])
                    link_anuncio.append(anuncio)
                except:
                    pass
    return fechas, link_img, titulo_anuncio, precio_anuncio, descripcion_anuncio, link_anuncio, estado_articulo, tipo_articulo

<div class="alert alert-info">Define the lists containing the information of the image url, post title, post description, price of the article and url of the post.</div>

In [14]:
ALL_dates = []
ALL_links = []
ALL_images = []
ALL_titles = []
ALL_prices = []
ALL_descriptions = []
ALL_estados = []
ALL_types = []

<div class="alert alert-info">Execute the loop to gather the information of the listings with all possible combinations of subtypes and states of the bike.</div>

In [ ]:
for opciones in suboptions:
    for states in estados:
        #select subcategory and state of the article
        select_subcategoria_and_estado(opciones, states, estados_xpath)
        #load more listings
        load_more_products()
        #get the html code of the webpage
        soup = bs4.BeautifulSoup(driver.page_source, 'html.parser')
        #get all the info of the listings
        output = get_info_listing(soup)
        #append all data gathered
        ALL_dates = ALL_dates + output[0]
        ALL_links = ALL_links + output[5]
        ALL_images = ALL_images + output[1]
        ALL_titles = ALL_titles + output[2]
        ALL_prices = ALL_prices + output[3]
        ALL_descriptions = ALL_descriptions + output[4]
        ALL_estados = ALL_estados + output[6]
        ALL_types = ALL_types + output[7]
        #deselect subcategory and state of the article
        deselect_options()

<div class="alert alert-info">Create a DataFrame with the previous information.</div>

In [17]:
df = pd.DataFrame(data = {'Link': ALL_links, 'Title':ALL_titles,
                  'Description': ALL_descriptions, 'Price': ALL_prices,
                 'Image': ALL_images, 'Date': ALL_dates, 'Type': ALL_types,
                          'State': ALL_estados})

In [18]:
df

,Link,Title,Description,Price,Image,Date,Type,State
0,https://es.wallapop.com/item/bicicleta-de-pase...,Bicicleta de paseo plegable,La bicicleta está nueva no tiene ningún defect...,255.0,https://cdn.wallapop.com/images/10420/e5/09/__...,2022-12-04,Bicicletas plegables,Nuevo
1,https://es.wallapop.com/item/bicicleta-estatic...,Bicicleta estática,Nueva,120.0,https://cdn.wallapop.com/images/10420/e4/wu/__...,2022-12-04,Bicicletas plegables,Nuevo
2,https://es.wallapop.com/item/bicicleta-plegabl...,bicicleta plegable,bicicleta plegable,60.0,https://cdn.wallapop.com/images/10420/e4/qe/__...,2022-12-05,Bicicletas plegables,Nuevo
3,https://es.wallapop.com/item/bicicleta-leotron...,bicicleta leotron energia,"pegable,nueva a estrenar. como indica en las i...",150.0,https://cdn.wallapop.com/images/10420/e4/d7/__...,2022-12-05,Bicicletas plegables,Nuevo
4,https://es.wallapop.com/item/monty-bici-plegab...,Monty bici plegable,Nueva sin uso se pliega todo hasta los pedales...,200.0,https://cdn.wallapop.com/images/10420/e4/a7/__...,2022-11-30,Bicicletas plegables,Nuevo
...,...,...,...,...,...,...,...,...
876,https://es.wallapop.com/item/bicicleta-rockrid...,Bicicleta Rockrider más rodillo Elite Qubo Fluid,Vendo Bicicleta Rockrider 120 talla L más rodi...,300.0,https://cdn.wallapop.com/images/10420/e4/14/__...,2022-12-03,MTB,Bueno
877,https://es.wallapop.com/item/bicicleta-rockrid...,Bicicleta Rockrider,"Bicicleta Rockrider de aluminio, XL, cambios S...",85.0,https://cdn.wallapop.com/images/10420/e3/ap/__...,2022-12-01,MTB,Bueno
878,https://es.wallapop.com/item/bicicleta-mtb-ktm...,bicicleta mtb KTM Ultra Flite,"KTM Ultra Flite 26"". Frenos hidráulicos Shiman...",309.0,https://cdn.wallapop.com/images/10420/e3/ru/__...,2022-12-03,MTB,Bueno
879,https://es.wallapop.com/item/portabidon-btt-az...,Portabidon btt azul,Portabidon bicicleta montaña nylon azul. En p...,3.0,https://cdn.wallapop.com/images/10420/e3/kn/__...,2022-11-26,MTB,Bueno


### Part 2

The goal of this part is to add new columns in the data set like the size and if it is a bike for children.

<div class="alert alert-info">Drop duplicates if there is any.</div>

In [19]:
df.drop_duplicates(subset=['Link'], inplace=True)

<div class="alert alert-info">Check if the post mentions that the bike is for children or the size of the bike and store them in new columns in the DataFrame</div>

In [20]:
infant = {'niño/a', 'niño', 'niña', 'niños', 'niñas', 'niño/as'}
letters = {'S', 'M', 'L', 'M/L', 'M-L', 'XL'}
child = []
tallas = []
tallas_letra = []
for i in range(len(df)):
    junto = df.iloc[i]['Title'].lower() + ' ' + df.iloc[i]['Description'].lower()
    if any(ele in junto for ele in infant):
        child.append(True)
    else:
        child.append(np.nan)
    if 'talla' in junto:
        size = junto.split('talla')[1].split()[0].upper()
        size = re.sub(u",", '', size)
        size = re.sub("\.", '', size)
        size = size.replace("CM", "" )
        try:
            tallas.append(int(size))
        except:
            tallas.append(np.nan)
        if any(ele == size for ele in letters):
            tallas_letra.append(size)
        else:
            tallas_letra.append(np.nan)
    else:
        tallas.append(np.nan)
        tallas_letra.append(np.nan)
        
df['Children'] = child
df['Size'] = tallas
df['Size (letter)'] = tallas_letra

In [21]:
df

,Link,Title,Description,Price,Image,Date,Type,State,Children,Size,Size (letter)
0,https://es.wallapop.com/item/bicicleta-de-pase...,Bicicleta de paseo plegable,La bicicleta está nueva no tiene ningún defect...,255.0,https://cdn.wallapop.com/images/10420/e5/09/__...,2022-12-04,Bicicletas plegables,Nuevo,NaN,NaN,NaN
1,https://es.wallapop.com/item/bicicleta-estatic...,Bicicleta estática,Nueva,120.0,https://cdn.wallapop.com/images/10420/e4/wu/__...,2022-12-04,Bicicletas plegables,Nuevo,NaN,NaN,NaN
2,https://es.wallapop.com/item/bicicleta-plegabl...,bicicleta plegable,bicicleta plegable,60.0,https://cdn.wallapop.com/images/10420/e4/qe/__...,2022-12-05,Bicicletas plegables,Nuevo,NaN,NaN,NaN
3,https://es.wallapop.com/item/bicicleta-leotron...,bicicleta leotron energia,"pegable,nueva a estrenar. como indica en las i...",150.0,https://cdn.wallapop.com/images/10420/e4/d7/__...,2022-12-05,Bicicletas plegables,Nuevo,NaN,NaN,NaN
4,https://es.wallapop.com/item/monty-bici-plegab...,Monty bici plegable,Nueva sin uso se pliega todo hasta los pedales...,200.0,https://cdn.wallapop.com/images/10420/e4/a7/__...,2022-11-30,Bicicletas plegables,Nuevo,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
876,https://es.wallapop.com/item/bicicleta-rockrid...,Bicicleta Rockrider más rodillo Elite Qubo Fluid,Vendo Bicicleta Rockrider 120 talla L más rodi...,300.0,https://cdn.wallapop.com/images/10420/e4/14/__...,2022-12-03,MTB,Bueno,NaN,NaN,L
877,https://es.wallapop.com/item/bicicleta-rockrid...,Bicicleta Rockrider,"Bicicleta Rockrider de aluminio, XL, cambios S...",85.0,https://cdn.wallapop.com/images/10420/e3/ap/__...,2022-12-01,MTB,Bueno,NaN,NaN,NaN
878,https://es.wallapop.com/item/bicicleta-mtb-ktm...,bicicleta mtb KTM Ultra Flite,"KTM Ultra Flite 26"". Frenos hidráulicos Shiman...",309.0,https://cdn.wallapop.com/images/10420/e3/ru/__...,2022-12-03,MTB,Bueno,NaN,NaN,S
879,https://es.wallapop.com/item/portabidon-btt-az...,Portabidon btt azul,Portabidon bicicleta montaña nylon azul. En p...,3.0,https://cdn.wallapop.com/images/10420/e3/kn/__...,2022-11-26,MTB,Bueno,NaN,NaN,NaN


In [22]:
#df.to_csv('out.csv', index=False)

<div class="alert alert-info">Create a new dataframe called agg that includes the aggregated data for the "Type" and "State" columns. This Dataframe should include the average prices by "Type" and "State" (in this order).</div>

In [23]:
agg = df.groupby(['Type', 'State']).agg(avg_Price=('Price','mean')).reset_index()

In [ ]:
agg

### Bonus

The goal of this part is to retrieve the date when the post was published.

<div class="alert alert-warning">
    The Bonus was completed inside <b>Part 1</b>.
</div>